<a href="https://colab.research.google.com/github/taukappawho/nets_mod/blob/main/notebook8eb496cfb7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks

## Project: Write an Algorithm for a Dog Identification App

---

In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the Jupyter Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that our TA can see the final implementation and output. You can then export the notebook by using the menu above and navigating to **File -> Download as -> HTML (.html)**. Or, you could print your result in a PDF by using the menu above and navigating to **File -> Print -> Save as PDF**. Include the finished document along with this notebook as your submission.

In addition to implementing code in Part B, there will be follow-up questions that you must answer in Part C which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Part C Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut.  Markdown cells can be edited by double-clicking the cell to enter edit mode.

The rubric contains _optional_ "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. If you decide to pursue the "Stand Out Suggestions", you should include the code in this Jupyter notebook.



---
### Why We're Here

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, your code will accept any user-supplied image as input.  If a dog is detected in the image, it will provide an estimate of the dog's breed.  If a human is detected, it will provide an estimate of the dog breed that is most resembling.  The image below displays potential sample output of your finished project (... but we expect that each student's algorithm will behave differently!).

![Sample Dog Output](https://www.dropbox.com/s/ckx8gitx39onqbc/sample_dog_output.png?dl=1)

In this real-world setting, you will need to piece together a series of models to perform different tasks; for instance, the algorithm that detects humans in an image will be different from the CNN that infers dog breed.  There are many points of possible failure, and no perfect algorithm exists.  Your imperfect solution will nonetheless create a fun user experience!

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Detect Humans
* [Step 2](#step2): Detect Dogs
* [Step 3](#step3): Create a CNN to Classify Dog Breeds (from Scratch)
* [Step 4](#step4): Create a CNN to Classify Dog Breeds (using Transfer Learning)
* [Step 5](#step5): Write your Algorithm
* [Step 6](#step6): Test Your Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets


Run the following 4 commands to download the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dogImages`.

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home diretcory, at location `/lfw`.  

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

In [ ]:
!unzip dogImages.zip

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip

In [ ]:
!unzip lfw.zip

In [ ]:
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("lfw/*/*"))
dog_files = np.array(glob("dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

<a id='step1'></a>
## Step 1: Detect Humans

In this section, we use OpenCV's implementation of [Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html) to detect human faces in images.  

OpenCV provides many pre-trained face detectors, stored as XML files on [github](https://github.com/opencv/opencv/tree/master/data/haarcascades).  We have downloaded one of these detectors and stored it in the `haarcascades` directory.  In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [ ]:
# NOTICE: If you run locally, the file `haarcascade_frontalface_alt.xml` is ready to use
# in the start code. You DO NOT need to run the following command to re-download it again!

# Download haarcascade_frontalface_alt.xml
!mkdir haarcascades
!wget -P haarcascades/ https://www.dropbox.com/s/dy70c8t97775y6i/haarcascade_frontalface_alt.xml

In [ ]:
%pip install opencv-python
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# load color (BGR) image
img = cv2.imread(human_files[0])
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find faces in image
faces = face_cascade.detectMultiScale(gray)

# print number of faces detected in the image
print('Number of faces detected:', len(faces))

# get bounding box for each detected face
for (x,y,w,h) in faces:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

Before using any of the face detectors, it is standard procedure to convert the images to grayscale.  The `detectMultiScale` function executes the classifier stored in `face_cascade` and takes the grayscale image as a parameter.  

In the above code, `faces` is a numpy array of detected faces, where each row corresponds to a detected face.  Each detected face is a 1D array with four entries that specifies the bounding box of the detected face.  The first two entries in the array (extracted in the above code as `x` and `y`) specify the horizontal and vertical positions of the top left corner of the bounding box.  The last two entries in the array (extracted here as `w` and `h`) specify the width and height of the box.

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### Assess the Human Face Detector

Use the code cell below to test the performance of the `face_detector` function. Then, you will see the results based on OpenCV detector
- The percentage of the first 100 images in `human_files` having a detected human face.
- The percentage of the first 100 images in `dog_files` having a detected human face.

Ideally, we would like 100% of human images with a detected face and 0% of dog images with a detected face.  You will see that our algorithm falls short of this goal, but still gives acceptable performance.  We extract the file paths for the first 100 images from each of the datasets and store them in the numpy arrays `human_files_short` and `dog_files_short`.

In [ ]:
%pip install tqdm
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

#-#-# Do NOT modify the code above this line. #-#-#

## Test the performance of the face_detector algorithm
## on the images in human_files_short and dog_files_short.
human_in_human_imgs = 0
human_in_dog_imgs = 0

for human in human_files_short:
    if face_detector(human):
        human_in_human_imgs +=1

for dog in dog_files_short:
    if face_detector(dog):
        human_in_dog_imgs +=1

print('%d%% of human images detected a face.' % human_in_human_imgs)
print('%d%% of dog images detected a face.' % human_in_dog_imgs)


Here, we use the face detector from OpenCV as a potential way to detect human images in your algorithm, but you are free to explore other approaches, especially approaches that make use of deep learning :).

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

In [ ]:
%pip install torch
import torch
%pip install torchvision
import torchvision.models as models
torch.manual_seed(0)

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

Given an image, this pre-trained VGG-16 model returns a prediction (derived from the 1000 possible categories in ImageNet) for the object that is contained in the image.

### Making Predictions with a Pre-trained Model

In the next code cell, you will see a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained VGG-16 model.  The output should always be an integer between 0 and 999, inclusive.

You could take the time to learn how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to
    predicted ImageNet class for image at specified path

    Args:
        img_path: path to an image

    Returns:
        Index corresponding to VGG-16 model's prediction
    '''

    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image

    # All pre-trained models expect input images normalized in the same way,
    # i.e. mini-batches of 3-channel RGB images of shape (3 x H x W),
    # where H and W are expected to be at least 224.
    # The images have to be loaded in to a range of [0, 1]
    # and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    normalize])

    # Open image and apply transformation
    image = Image.open(img_path)
    image = transform(image)
    image.unsqueeze_(0)

    # move image to GPU if CUDA is available
    if use_cuda:
        image = image.cuda()

    # Predict using VGG16
    image_var = torch.autograd.Variable(image)
    prediction = VGG16(image_var)
    value, index = torch.max(prediction, 1)

    return index[0].item() # predicted class index

### Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    ## TODO: Complete the function.
    prediction_index = VGG16_predict(img_path)
    return (151 <= prediction_index <= 268) # true/false

### Assess the Dog Detector use a Pre-trained CNN

Use the code cell below to test the performance of your `dog_detector` function. It will indicate:  
- The percentage of the images in `human_files_short` having a detected dog.  
- The percentage of the images in `dog_files_short` having a detected dog.

In [ ]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dog_in_human_imgs = 0
dog_in_dog_imgs = 0

for human in human_files_short:
    if dog_detector(human):
        dog_in_human_imgs +=1

for dog in dog_files_short:
    if dog_detector(dog):
        dog_in_dog_imgs +=1

print('%d%% of human images detected as a dog.' % dog_in_human_imgs)
print('%d%% of dog images detected as a dog.' % dog_in_dog_imgs)

As you can see, the detection result is perfect by using pre-trained CNNs. We suggest VGG-16 as a potential network to detect dog images in your algorithm, but you are free to explore other pre-trained networks (such as [Inception-v3](http://pytorch.org/docs/master/torchvision/models.html#inception-v3), [ResNet-50](http://pytorch.org/docs/master/torchvision/models.html#id3), etc).

---

### ✅ Notice: You need to start your coding from Step 3 to Step 6, which is required in **Part B**.  Sections that begin with **'(IMPLEMENTATION)'** in the header indicate that the following block of code will require additional functionality which you must provide.

### 💁 Tips: For all the missing fucntions you need to implement in **Part B**, you could check our in-class exercise code for references. Try to modify the code and fit it into the problem here based on your understanding.

### ✅ Also There are some follow-up questions associated with coding, highlighted with **Part C Question X**. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**.

### ✅ Your project submission will be evaluated based on your answers to each of the questions for **Part C** and the code implementation you provide as your answer for **Part B**.




---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least **10%**.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging. To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | -
<img src="https://www.dropbox.com/s/bo3mwqju253alon/Brittany_02625.jpg?dl=1" width="100"> | <img src="https://www.dropbox.com/s/lb2hzvq2m5y44rg/Welsh_springer_spaniel_08203.jpg?dl=1" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="https://www.dropbox.com/s/p65g938k8q0kfbf/Curly-coated_retriever_03896.jpg?dl=1" width="200"> | <img src="https://www.dropbox.com/s/b9zv9v6iv16f4l1/American_water_spaniel_00648.jpg?dl=1" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | - | -
<img src="https://www.dropbox.com/s/nni1x0fvxu8apqu/Labrador_retriever_06457.jpg?dl=1" width="150"> | <img src="https://www.dropbox.com/s/7ik3skchbd89yas/Labrador_retriever_06455.jpg?dl=1" width="240"> | <img src="https://www.dropbox.com/s/64fboclmef83zd1/Labrador_retriever_06449.jpg?dl=1" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION [5 points]) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
import os
from torchvision import datasets

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
# define dataloader parameters
batch_size = 32

from torch.utils.data import DataLoader
from PIL import ImageFile  # Import this to handle truncated images

# ✅ Add this line to allow loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Tranformation with data augmentation to be used on the training set
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # Flip image horizontally with 50% probability
    transforms.RandomRotation(15),  # Rotate image within ±15 degrees
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet mean/std
])

# Tranformation with data augmentation to be used on the validation and test set
test_transform =  transforms.Compose([
    transforms.Resize(256),  # Resize to 256 pixels on shortest side
    transforms.CenterCrop(224),  # Crop the center to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Defining the datasets for training, validation and testing
train_data = datasets.ImageFolder('dogImages/train', transform=train_transform)
valid_data = datasets.ImageFolder('dogImages/valid', transform=test_transform)
test_data = datasets.ImageFolder('dogImages/test', transform=test_transform)

# Defining the loaders for training, validation and testing
loaders_scratch={
    'train': torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True),
    'valid': torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True),
    'test': torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
}

**Part C Question 1 [5 points]:** Describe your chosen procedure for preprocessing the data.
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:

### (IMPLEMENTATION [5 points]) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN

        # Convolutional layers
        # self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        # self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)

        # # Batch normalization
        # self.bn1 = nn.BatchNorm2d(32)
        # self.bn2 = nn.BatchNorm2d(64)
        # self.bn3 = nn.BatchNorm2d(128)

        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Flatten from convolutional layers
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 133)  # Output layer

        # Dropout
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Convolutional layers with batch normalization, activation, and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output for the fully connected layers
        # x = x.view(x.size(0), -1)  # Flatten
        x = x.view(-1, 128 * 28 * 28)

        # Fully connected layers with dropout and activation
        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        # Output layer (Softmax is applied automatically in CrossEntropyLoss)
        x = self.fc3(x)

        return x


#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

__Part C Question 2 [5 points]:__ Outline your final CNN architecture (from scratch) in Step 3.  
Hint: you could use `print(model_scratch)` to get the CNN architecture.

In [ ]:
print(model_scratch)

__Answer:__

### (IMPLEMENTATION [5 points]) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = torch.nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch =  optim.Adam(model_scratch.parameters(), lr=0.001)

### (IMPLEMENTATION [10 points]) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.inf

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## TODO:
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))


            optimizer.zero_grad()  # Clear previous gradients

            ## Forward pass: Compute predictions
            outputs = model(data)

            ## Compute the loss
            loss = criterion(outputs, target)

            ## Backward pass: Compute gradients and update model parameters
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model weights

            ## Record average training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

        ######################
        # validate the model #
        ######################
        model.eval()
        # Disable gradient calculation for validation
        with torch.no_grad():
          for batch_idx, (data, target) in enumerate(loaders['valid']):
              # move to GPU
              if use_cuda:
                  data, target = data.cuda(), target.cuda()
              ## TODO:
              ## update the average validation loss
              ## record the average validation loss, using something like
              ## valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

              # Forward pass
              output = model(data)

              # Compute loss
              loss = criterion(output, target)

              # Update the average validation loss
              valid_loss += (1 / (batch_idx + 1)) * (loss.item() - valid_loss)

        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    # return trained model
    return model


# train the model
model_scratch = train(100, loaders_scratch, model_scratch, optimizer_scratch,
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

### (PERFORMENCE EVALUATION [5 points]) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater equal than 10%.

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)

    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively).

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

In [ ]:
## Specify data loaders (we use the same data loader created in step 3)
loaders_transfer = loaders_scratch.copy()

### (IMPLEMENTATION [10 points]) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
import torchvision.models as models
import torch.nn as nn

## TODO: Specify model architecture
## name the model as `model_transfer`
# Load the pre-trained network densenet161
model_transfer = models.densenet161(pretrained=True)

# Freeze training for all "features" layers
for param in model_transfer.parameters():
    param.requires_grad = False

# replace the last fully connected layer with a Linnear layer 133 output
# Replace the classifier layer with a new Linear layer (output = 133 classes)
num_ftrs = model_transfer.classifier.in_features  # Get input features of last layer
model_transfer.classifier = nn.Linear(num_ftrs, 133)  # Replace classifier

if use_cuda:
    model_transfer = model_transfer.cuda()

__Part C Question 3 [5 points]:__ Outline your final CNN architecture (using transfer learning) in Step 4.

Hint: you could use `print(model_transfer)` to get the CNN architecture.

In [ ]:
print(model_transfer)

__Answer:__


### (IMPLEMENTATION [5 points]) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:
criterion_transfer = torch.nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_transfer, step_size=5, gamma=0.5)

### (IMPLEMENTATION [5 points]) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
model_transfer = train(40, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (PERFORMENCE EVALUATION [5 points + (possibly) 10 bonus point]) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

You also have the possibility to get a total of 10 additional bonus points if you obtain high test accuracy after your parameter's turning. The rubric is as follows.

If your test accuracy under transfer learning is | You will get
- | -
above 90% | 10 bonus points
above 80% but below 90% |5 bonus points
below 80% | 0 bonus points

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### The function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in train_data.classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed

    # All pre-trained models expect input images normalized in the same way,
    # i.e. mini-batches of 3-channel RGB images of shape (3 x H x W),
    # where H and W are expected to be at least 224.
    # The images have to be loaded in to a range of [0, 1]
    # and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    normalize])

    # Open image and apply transformation
    image = Image.open(img_path)
    image = transform(image)
    image.unsqueeze_(0)

    # move image to GPU if CUDA is available
    if use_cuda:
        image = image.cuda()

    # Predict using model_transfer
    image_var = torch.autograd.Variable(image)
    prediction = model_transfer(image_var)
    value, index = torch.max(prediction, 1)

    return class_names[index[0].item()]

---
<a id='step5'></a>
## Step 5: Single Image Prediction

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `human_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](https://www.dropbox.com/s/53a4v25xecb4iz1/sample_human_output.png?dl=1)


### A Fuction Called `run_app` Being Ready for Single Image Prediction

In [ ]:
### Write your algorithm.
### Feel free to use as many code cells as needed.
from PIL import Image
import requests
from io import BytesIO

def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    img = Image.open(img_path)

    if dog_detector(img_path):
        plt.imshow(img)
        plt.show()
        print('Hello, dog!')
        print('Your predicted breed is ...', predict_breed_transfer(img_path))
    elif face_detector(img_path):
        plt.imshow(img)
        plt.show()
        print('Hello, human!')
        print('Your look like a ...', predict_breed_transfer(img_path))
    else:
        plt.imshow(img)
        plt.show()
        print('Classification error, no dog or human found!')

---
<a id='step6'></a>
## Step 6: Test Your Model

In this section, you will take your new model for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION [5 points]) Test Your Model on Sample Images!

Test your model at least six images on your computer. Feel free to use any images you like. Use at least two human and two dog images.  

__Part C Question 4 [5 points]:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least TWO possible points of improvement for your algorithm.

__Answer:__ (TWO possible points for improvement)

1.
2.

In [ ]:
## TODO: Execute your model from Step 4 on
## at least 6 images on your computer by calling
## the `run_app` function you implement in Step 5.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)

## or you could creat a folder called test_images
## then load six your favorite images to this folder
## then run code below
for filename in os.listdir('./test_images/'):
    if filename.endswith(".jpg"):
        run_app('./test_images/' + filename)